<a href="https://colab.research.google.com/github/xanderkov/DL_Subjets/blob/main/F_%D0%9A%D0%BE%D0%B2%D0%B5%D0%BB%D1%8C_%D0%90%D0%BB%D0%B5%D0%BA%D1%81%D0%B0%D0%BD%D0%B4%D1%80_%D0%94%D0%B5%D0%BD%D0%B8%D1%81%D0%BE%D0%B2%D0%B8%D1%87.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ковель Алексаднр Денисович

## Часть 1. Парсинг (10 баллов)


Мы предлагаем вам распарсить часть данных litres.ru — а именно часть про книги, имеющие отношение к программированию.

Они находятся в этом каталоге: https://www.litres.ru/genre/programmirovanie-5272/

Вам нужно собрать датасет о книгах с первых 40-ти страниц каталога (около 960 книг.) Подсказка. Можете начать с исследования html кода этой страницы.

Вам необходимо собрать датасет, содержащий следующие данные:

- name: название книги
- author: автор
- link: ссылка на книгу
- rating: рейтинг по 5-балльной шкале
- rating_count: количество оценок
- review_count: количество отзывов
- pages_count: объем (число страниц)
- price: цена
- text_reviews: тексты отзывов: список строк
- age: возрастное ограничение
- year: год написания
В дальнейшем вам нужно будет анализировать полученный у вас датасет.

Если у вас не получилось спарсить датасет, воспользуйтесь для последующих пунктов готовым датасетом: https://disk.yandex.ru/d/2UZet2-qxRxF6Q

Так вы сможете получить баллы за вторую часть.

(Датасет неидеален, с ним придется поработать!)

Парсите "вежливо": используйте time.sleep() не меньше 1-2 секунд между запросами к сайту!

In [133]:
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
from dataclasses import dataclass, asdict
from tqdm.notebook import tqdm
import time
import traceback
import unidecode


%matplotlib inline

ModuleNotFoundError: No module named 'unidecode'

In [123]:
@dataclass
class Book:
  name: str
  author: str
  link: str
  rating: float
  rating_count: int
  review_count: int
  pages_count: int
  price: float
  text_reviews: list[str]
  age: int
  year: int


In [124]:
litres_url = "https://www.litres.ru"
litres_url_books = 'https://www.litres.ru/genre/programmirovanie-5272/'

response = requests.get(litres_url_books)

soup = BeautifulSoup(response.text, 'html.parser')
N = 40
books_dataset: list[Book] = []
soup.title

<title>Лучшие книги в жанре Программирование – скачать или читать онлайн бесплатно на Литрес</title>

In [142]:
def get_current_book(current_book_url: str) -> Book | None:
  try:
    result = None
    book_response = requests.get(current_book_url)
    if book_response.status_code != 200:
      print(book_response.status_code)
      return None
    book_soup = BeautifulSoup(book_response.text, 'html.parser')
    name = book_soup.find('h1', {'class': 'BookCard_book__mainInfo__title___nFS4'})
    author = book_soup.find('div', {'class': 'Truncate_truncated__jKdVt'})
    ratings = book_soup.find('div', {'class': 'BookFactoids_section__59q_1'})
    rating = ratings.contents[2]
    rating_count = ratings.contents[3]
    review_count = book_soup.find('span', {'class', 'CommentSystem_commentsCount__woGh3'})
    pages_count = book_soup.find('div', {'class', 'BookCard_book__preview__data__XjF_j'})
    price = book_soup.find('div', {'class', 'SaleBlock_block__price__pC6XL'})
    text_reviews = book_soup.find_all('div', {'class', 'Comment_reviewText__PEkHn'})
    age_rating = pages_count
    table = book_soup.find_all('div', {'class', 'CharacteristicsBlock_characteristic__4pi7v'})
    year = table[1].contents[1].contents[0].split()[-1]


    result = Book(
          name=name.contents[0],
          author=author.div.span.contents[0],
          link=current_book_url,
          rating=float(rating.attrs["content"]),
          rating_count=int(rating_count.attrs["content"]),
          review_count=int(review_count.contents[0]) if review_count else 0,
          pages_count=int(pages_count.p.contents[0].split()[1]),
          price=float(price.strong.contents[0].replace("&nbsp", "").replace(u"\xa0", "")[:-1]),
          text_reviews=[t.p.contents[0] for t in text_reviews if t.p] if text_reviews else 0,
          age=int(age_rating.contents[-1].contents[0][:-1]),
          year=int(year)
    )
  except Exception as e:
    print(price.strong.contents[0].replace("&nbsp", "").replace(" ", ""))
    print(e)
    print(traceback.format_exc())
    print(current_book_url)
  return result


books = soup.find_all('div', {'class': 'ArtDefault_cover__text__HKF_g'})
for page in tqdm(range(N), desc="Books Pages"):
  for book in tqdm(books, desc="By book", leave=False):
    link = book.find('div', {'class': 'ArtInfo_wrapper__GoMsb'})
    book_url = litres_url + link.a['href']
    time.sleep(0.1)
    book_d = None
    if 'audio' not in book_url:
      book_d = get_current_book(book_url)
    if book_d:
      books_dataset.append(book_d)
  next_link = f"{litres_url_books}?page={page + 1}"
  response = requests.get(next_link)
  soup = BeautifulSoup(response.text, 'html.parser')
  books = soup.find_all('div', {'class': 'ArtDefault_cover__text__HKF_g'})
  time.sleep(0.5)

Books Pages:   0%|          | 0/40 [00:00<?, ?it/s]

By book:   0%|          | 0/24 [00:00<?, ?it/s]

By book:   0%|          | 0/24 [00:00<?, ?it/s]

UnboundLocalError: local variable 'price' referenced before assignment

In [141]:
df = pd.DataFrame([b for b in books_dataset if b])
df.to_csv('cache.csv')
df.head(5)

,name,author,link,rating,rating_count,review_count,pages_count,price,text_reviews,age,year
0,Изучаем DDD – предметно-ориентированное проект...,Влад Хононов,https://www.litres.ru/book/vlad-hononov/izucha...,0.0,0,1,319,39.0,"[Эта книга из той лиги, в которой я пока не со...",0,2024
1,Высоконагруженные приложения. Программирование...,Мартин Клеппман,https://www.litres.ru/book/martin-kleppman-173...,4.0,51,11,640,799.0,"[Не могу что-либо про перевод сказать, читал д...",16,2018
2,Начинаем программировать на Python,Тони Гэддис,https://www.litres.ru/book/toni-geddis-3249116...,5.0,16,6,875,39.0,[Шикарная книга! Начал изучать Python самостоя...,12,2023
3,System Design. Подготовка к сложному интервью,Алекс Сюй,https://www.litres.ru/book/aleks-suy/system-de...,3.6,17,9,304,39.0,[Добротная книга по разработке архитектуры при...,16,2022
4,Нейросети начало,Джейд Картер,https://www.litres.ru/book/dzheyd-karter/neyro...,4.9,439,37,102,39.0,[Книга отличная. Давно хотела понять как все р...,12,2023


## Часть 2. EDA (30 баллов)




### 1. Выведите первые 5 строк датасета. (0.25). Сколько в нём строк и столбцов (0.25)?

### 2. Есть ли в датасете пропуски? (0.5)


### 3. Проверьте типы данных. Если это необходимо, приведите к типам int и float те столбцы, с которыми понадобится работать как с числами. (1).

### 4. Выведите описательные статистики переменных. Ответьте на следующие вопросы:

* Какая медианная цена книги в вашем датасете? (1)  
* Какое возрастное ограничение встречается чаще всего? (1)  
* Какое среднее число отзывов в книге? (1)  
* Сколько книг имеют оценку ниже 4.25? (1)  
* В каком году было написано больше всего книг из датасета? (1)  

### 5. Если вы работаете с готовым датасетом, то попробуйте "достать" из столбца pages количество страниц. Если у вас не получилось, то далее при определении числа страниц пользуйтесь стольцов pages_count. Если вы парсили датасет сами, то вы получаете балл за этот пункт автоматически (1.5)

### 6. Создайте новое поле is_popular. Значение равно 1, если рейтинг книги не менее 4.6 и при этом у нее не менее 5 отзывов, и 0 в остальных случаях. (1)

### 7. Как отличается среднее число страниц среди популярных и непопулярных книг? (2)

### 8. Выведите топ-10 книг по числу отзывов. (2).



### 9. Найдите среднюю длину отзыва (в символах). (2)


### 10. Постройте таблицу корреляций числовых переменных. (1) Прокомментируйте результаты. (1)


### 11. Постройте диаграмму рассеяния (scatterplot) количества страниц и количества отзывов. Не забудьте подписать график и оси. (1) Прокомментируйте полученные результаты. (1)


### 12. Постройте линейный график: по оси Х год, по оси Y количество книг. (1) Прокомментируйте. (1)


### 12. Постройте еще любые два графика по вашему усмотрению. (2) Прокомментируйте полученные результаты. (1.5)

### 13. Постройте таблицу с авторами книг с именем автора, количество книг в датасете, средней оценкой книг, средним количеством отзывов. (2).

### 14. Что еще интересного можно увидеть в этом датасете? Просмотрите на данные и ответьте на какие-нибудь вопросы, на которые не ответили в предыдущим пункте. Мы никак не ограничиваем вашу фантазию! (3).